# Making Inputs
To use any model in BatMods-lite, you need to know how to make inputs. There are two types of inputs for each model:

* simulation inputs -> a ``.yaml`` file
* experiment inputs -> a Python dictionary

Simulation inputs define battery parameters and experimental inputs define the details of an experiment. To get help with either of these inputs, use the ``templates()`` function. Each BatMods-lite model subpackage has its own ``templates()`` function, so to see pseudo-2D model (P2D) templates, we will call the function from the ``P2D`` package, as shown below. When no arguments are given to ``templates()``, a list of available templates will be printed.

In [2]:
import bmlite as bm

bm.P2D.templates()


Templates for P2D simulations:
- [0] graphite_nmc532

Templates for P2D experiments:
- [0] constant_current
- [1] constant_power
- [2] constant_voltage


You can view the contents of any template by passing arguments to ``templates()``. The first argument is the name (``str``) or index (``int``) of the desired simulation template, and the second argument is the name or index of the experiment. For example, to see the "graphite_nmc532" and "constant_current" templates, run either of the following:

```python
bm.P2D.templates('graphite_nmc532', 'constant_current')
bm.P2D.templates(0, 0)
```

If you don't need to request both a simulation and experimental template at the same time, you can specify that you only want one or the other by using named arguments, for example ``templates(sim=0)`` or ``templates(exp=0)``. 

## Simulation inputs
Let's take a look at the output for the "graphite_nmc532" simulation template, below.

In [3]:
bm.P2D.templates(sim='graphite_nmc532')


graphite_nmc532.yaml

battery:
    cap:        1.89e-2         # Nominal capacity                       [A*h]
    temp:       303.15          # Temperature                              [K]
    area:       1.4e-3          # Cell area                              [m^2]

electrolyte:
    Li_0:       1.2             # Initial Li+ conc.                 [kmol/m^3]
    material:   Gen2Electrolyte # Electrolyte material class               [-]

anode:
    Nx:         32              # Number of x discretizations              [-]
    Nr:         30              # Number of r discretizations              [-]
    thick:      44.0e-6         # Thickness                                [m]
    R_s:        4.0e-6          # Secondary particle radius                [m]
    eps_s:      0.6             # Solid-phase volume frac.                 [-]
    eps_el:     0.4             # Electrolyte volume frac.                 [-]
    eps_CBD:    0.0569272237    # Carbon-binder-domain volume frac.        [-

The output above shows a full list of parameters that must be specified in a P2D ``.yaml`` file. To make your own ``.yaml`` file, it is recommended to copy the template contents into your own file and then edit the values as needed. 

You can initialize a simulation with your parameters by using the ``bm.P2D.Simulation`` class. For example, with your new file ``'newfile.yaml'``, call

```python
sim = bm.P2D.Simulation('newfile.yaml')
```

Note that the ``.yaml`` extension is optional. Also, the input should be the absolute or relative path to your ``.yaml`` file, so if you save the file to a location outside your current working directory, you will have to modify the code above to point to the actual file location. You can also always construct a simulation with the default ``'graphite_nmc532.yaml'`` parameters by leaving the input blank, i.e. ``sim = bm.P2D.Simulation()``. A warning will print if you use any of the default files.

You can learn more about any of P2D parameters from the ``bm.P2D.domains`` module. You will also want to look at the ``bm.materials`` package to view all of the available material classes. Specifying an electrode active material that is not available in ``bm.materials`` will result in an error.

## Experiment inputs
Now, let's take a look at the ``'constant_current'`` template.

In [3]:
bm.P2D.templates(exp='constant_current')


constant_current.yaml
exp = {
    "C_rate": -2.0,
    "t_min": 0.0,
    "t_max": 1350.0,
    "Nt": 150
}


The ``'constant_current'`` template above displays a standard Python dictionary. The key/value pairs in each experiment template vary based on the specific experiment.  

## Important notes
You can modify simulation parameters after you read in a ``.yaml`` file, but doing so requires an extra step. A pre-processor automatically runs to determines the mesh, pointers, and bandwidth of the problem during the class initialization. However, if any parameters are changed, the dependent calculations may no longer be consistent with the new values. Therefore, if you change any parameters, you need to manually re-run the pre-processor. For example, 

```python
sim.an.thick = 40.e-6
sim.pre()
```

changes the anode thickness, and then calls the ``pre()`` method to update the mesh, pointers, etc.